In [ ]:
import ipywidgets.widgets as widgets
from jetbot import Robot, Camera, bgr8_to_jpeg
from jetbot import Camera

import traitlets

import uuid
import subprocess

#  Example using steer and drive

Rather than using left and right motor like before, we want to use a drive and steer knob.

In [ ]:
controller = widgets.Controller(index=0)  # replace with index of your controller
display(controller)

Creating robot and camera instances like before.

In [ ]:
robot=Robot()
camera = Camera.instance()

In [ ]:
image = widgets.Image(format='jpeg', width=300, height=300)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

Here is the main difference. Since we cannot simply associate a joystick value to a motor value, we need to create a class that has the traits `lin` (speed forward/backward) and `ang` (steering). These get set by the joysticks. The `observe` function then calls `control_vel` everytime a value changes. This makes this control a bit less responsive than in the other example.

In [ ]:
class Vel(traitlets.HasTraits):
    lin = traitlets.Float(default_value=0.0)
    ang = traitlets.Float(default_value=0.0)

vel = Vel()
wheel_separation = 1.0

lin_link =  traitlets.dlink((controller.axes[1], 'value'), (vel, 'lin'), transform=lambda x: -0.5*x)
ang_link = traitlets.dlink((controller.axes[2], 'value'), (vel, 'ang'), transform=lambda x: -0.5*x)

def control_vel(change):
    robot.left_motor.value = vel.lin - (vel.ang * wheel_separation / 2);
    robot.right_motor.value  = vel.lin + (vel.ang * wheel_separation / 2);

vel.observe(control_vel, names=('lin', 'ang'))


Finally the same code to take a snapshot and display everything on the screen. Remember, you can create a separate output window :)

In [ ]:
subprocess.call(['mkdir', '-p', 'snapshots'])

snapshot_image = widgets.Image(format='jpeg', width=300, height=300)

def save_snapshot(change):
    # save snapshot when button is pressed down
    if change['new']:
        file_path = 'snapshots/' + str(uuid.uuid1()) + '.jpg'
        
        # write snapshot to file (we use image value instead of camera because it's already in JPEG format)
        with open(file_path, 'wb') as f:
            f.write(image.value)
            
        # display snapshot that was saved
        snapshot_image.value = image.value


controller.buttons[2].observe(save_snapshot, names='value')

display(widgets.HBox([image, snapshot_image]))
display(controller)